In [ ]:
import json
import pandas as pd
from glob import glob

In [ ]:
in_df = pd.read_excel(glob("CallSignSeriesRanges*.xlsx")[0]).rename({"Allocated to": "region"}, axis=1)
in_df

In [ ]:
in_df["start0"] = in_df["Series"].apply(lambda x: x[0])
in_df["start01"] = in_df["Series"].apply(lambda x: x[:2])
in_df["start012"] = in_df["Series"].apply(lambda x: x[:3])
in_df["stop0"] = in_df["Series"].apply(lambda x: x[6])
in_df["stop01"] = in_df["Series"].apply(lambda x: x[6:8])
in_df["stop012"] = in_df["Series"].apply(lambda x: x[6:9])

in_df

In [ ]:
in_df = pd.concat(
    [in_df[["Series", "region", "start0", "start01", "start012"]],
    in_df[["Series", "region", "stop0", "stop01", "stop012"]].rename({
        "stop0": "start0",
        "stop01": "start01",
        "stop012": "start012"
    }, axis=1)]
)

in_df

In [ ]:
## check if the list of countries for the first position is unique
## check if the list of countries for the first two positions is unique
## check for the first three positions
## i.e. anything beginning with 2 must be the UK

In [ ]:
unique_first_pos_set = set(in_df["start0"])
first_round_dict = {}
seen_starts_set = set()

for x in unique_first_pos_set:
    temp_df = in_df[in_df["start0"] == x].drop_duplicates("region")
    if len(temp_df) == 1:
        first_round_dict[x] = list(temp_df["region"])[0]
        seen_starts_set.add(x)

In [ ]:
sub_df = in_df[~in_df["start0"].isin(seen_starts_set)]

unique_first_two_pos_set = set(sub_df["start01"])
second_round_dict = {}
seen_starts_second_set = set()

for x in unique_first_two_pos_set:
    temp_df = sub_df[sub_df["start01"] == x].drop_duplicates("region")
    if len(temp_df) == 1:
        second_round_dict[x] = list(temp_df["region"])[0]
        seen_starts_second_set.add(x)

In [ ]:
sub_df = sub_df[~sub_df["start01"].isin(seen_starts_second_set)]

unique_first_three_pos_set = set(sub_df["start012"])
third_round_dict = {}
seen_starts_third_set = set()

for x in unique_first_three_pos_set:
    temp_df = sub_df[sub_df["start012"] == x].drop_duplicates("region")
    if len(temp_df) == 1:
        third_round_dict[x] = list(temp_df["region"])[0]
        seen_starts_third_set.add(x)

In [ ]:
first_round_dict

In [ ]:
second_round_dict

In [ ]:
third_round_dict

In [ ]:
sub_df = sub_df[~sub_df["start01"].isin(seen_starts_third_set)]

sub_df = sub_df.drop_duplicates(["Series", "region"]).drop(["start0", "start01", "start012"], axis=1)

sub_df

In [ ]:
def parse_into_regex(inputRange):
    start,stop = inputRange.split(" - ")
    
    ## assert that the start and stop are always
    ## 3 characters long
    assert len(start) == 3, inputRange
    assert len(stop) == 3, inputRange
    ## assert that the first two characters of
    ## start and stop are always the same
    assert start[:2] == stop[:2], inputRange
    ## assert that the last character is always different
    assert start[-1] != stop[-1], inputRange
    
    ## if the last characters are A and Z...
    if (start[-1] == "A") and (stop[-1] == "Z"):
        ## assume that we only need to check the first two characters
        start = start[:-1]
        stop = stop[:-1]
        
    ## well if all that is the case, then we
    ## can easily build a regex, can't we?
    invariant_prefix = start[:2]
    variable_suffix_start = start[-1]
    variable_suffix_end = stop[-1]
    regex = f"^{invariant_prefix}[{variable_suffix_start}-{variable_suffix_end}]" if len(start) > 2 else f"^{invariant_prefix}"
    
    return regex

def parseStartStop(inputString):
    return inputString.split(" - ")

sub_df["regex"] = sub_df["Series"].apply(parse_into_regex)

sub_df

In [ ]:
## netherlands are badly behaved, fix that
dupes_df = sub_df[sub_df.duplicated(["Series"], keep=False)]
dupes_df = pd.concat([pd.DataFrame(
        [(" or ".join(list(temp_df["region"])), regex)]
    ) for regex, temp_df in dupes_df.groupby("regex")]).rename({0: "region", 1: "regex"}, axis=1)

dupes_df

In [ ]:
sub_df = sub_df.drop_duplicates("Series", keep=False).drop("Series", axis=1)
sub_df

In [ ]:
regex_records = []

for round_dict in [first_round_dict, second_round_dict, third_round_dict]:
    for k,v in round_dict.items():
        regex_records.append((v, f"^{k}"))
        
final_regex_df = pd.concat([
    pd.DataFrame(regex_records).rename({0: "region", 1: "regex"}, axis=1),
    sub_df,
    dupes_df
])

final_regex_df

In [ ]:
callsign_ref_dict = {}

for region, meta in final_regex_df.groupby("region"):
    callsign_ref_dict[region] = list(meta["regex"])

In [ ]:
with open("../res/callsign_ref.json", "w") as outfile:
    outfile.write(json.dumps(callsign_ref_dict, indent=4))